In [8]:
# import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
fig_dir = (Path().cwd().parents[0] / 'figure').absolute()

df_meta_path = data_dir / 'OCT mouse' / 'Whole' / 'metadata' /  'imgs_reg.csv'
df_imgs = pd.read_csv(df_meta_path)

# Read per cell data

In [12]:
# Read PPI dfs
PPI_save_path =  data_dir /  'OCT mouse' / 'Whole' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

# Group by location (Cell vs Nuclei)
g = df.groupby(['Condition', 'FOV', 'PPI', 'Cyto']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell = df_cell[df_cell.Cyto != 0]
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']

g = df.groupby(['Condition', 'FOV', 'PPI', 'Nuclei']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei= df_nuclei[df_nuclei.Nuclei != 0]
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']

# Concat 
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei'] # Cyto count

# Ranme columns
df_all.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Cell', 'Nuclei',
       'Cyto']

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,212841.000000,212841.000000,212841.000000,212841.000000
mean,7329.986018,6.973722,2.869795,4.103927
std,4945.882555,11.587545,5.525198,7.569558
min,1.000000,1.000000,0.000000,-29.000000
25%,3299.000000,2.000000,0.000000,1.000000
50%,6517.000000,3.000000,1.000000,2.000000
75%,10552.000000,8.000000,3.000000,5.000000
max,19720.000000,589.000000,176.000000,478.000000


,Cell,Nuclei,Cyto
count,61310.000000,61310.000000,61310.000000
mean,24.209656,9.962649,14.247007
std,32.264301,15.475948,21.256424
min,1.000000,0.000000,-112.000000
25%,6.000000,1.000000,4.000000
50%,15.000000,5.000000,9.000000
75%,31.000000,13.000000,18.000000
max,1932.000000,378.000000,1569.000000


In [13]:
# Filter out by maximum number of counts per cell
min_count = 5
max_count = 40

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cell'] < max_count).all())
# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,169331.000000,169331.000000,169331.000000,169331.000000
mean,7323.469323,6.114031,2.512605,3.601426
std,4952.326948,6.596182,3.640939,4.415188
min,1.000000,1.000000,0.000000,-29.000000
25%,3297.000000,2.000000,0.000000,1.000000
50%,6494.000000,4.000000,1.000000,2.000000
75%,10529.000000,8.000000,3.000000,5.000000
max,19720.000000,39.000000,56.000000,39.000000


,Cell,Nuclei,Cyto
count,44087.000000,44087.000000,44087.000000
mean,23.483000,9.650509,13.832490
std,17.092927,10.356716,10.981994
min,6.000000,0.000000,-112.000000
25%,11.000000,3.000000,6.000000
50%,18.000000,7.000000,11.000000
75%,31.000000,13.000000,19.000000
max,176.000000,246.000000,114.000000


# ML model

In [14]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
import wandb

In [15]:
condition = 'ML'
project_name = f'PLA_Tissue_051528_{condition}'

In [29]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count = df_count[~(df_count<0).any(axis=1)]
df_count_cell = df_count.iloc[:, df_count.columns.get_level_values(0)=='Cell']

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [30]:
df_count

Cell                                          
PPI                CylinE & CDK2 Mcl-1 & BAK P-ERK & c-MYC TEAD1 & YAP1   
Condition FOV Id                                                          
1M        FW1 1              4.0         0.0           0.0          2.0  \
              9              2.0         0.0           0.0          2.0   
              25             2.0         0.0           0.0          0.0   
              26             3.0         0.0           0.0          4.0   
              27             9.0         0.0           0.0          0.0   
...                          ...         ...           ...          ...   
1W        FW3 7018          10.0         0.0           5.0          1.0   
              7020           8.0         0.0          24.0          2.0   
              7022          23.0         0.0           9.0          3.0   
              7024          35.0         0.0          26.0          5.0   
              7025           3.0         0.0           0.0          6.0   

                                       Nuclei                             
PPI                p-AKT & mTOR CylinE & CDK2 Mcl-1 & BAK P-ERK & c-MYC   
Condition FOV Id                                                          
1M        FW1 1             1.0           0.0         0.0           0.0  \
              9             3.0           1.0         0.0           0.0   
              25            4.0           0.0         0.0           0.0   
              26            0.0           0.0         0.0           0.0   
              27            2.0           1.0         0.0           0.0   
...                         ...           ...         ...           ...   
1W        FW3 7018          2.0           0.0         0.0           0.0   
              7020          3.0           6.0         0.0           6.0   
              7022          5.0           2.0         0.0           2.0   
              7024          6.0          10.0         0.0          11.0   
              7025          2.0           0.0         0.0           0.0   

                                                      Cyto               
PPI                TEAD1 & YAP1 p-AKT & mTOR CylinE & CDK2 Mcl-1 & BAK   
Condition FOV Id                                                         
1M        FW1 1             0.0          0.0           4.0         0.0  \
              9             1.0          0.0           1.0         0.0   
              25            0.0          4.0           2.0         0.0   
              26            4.0          0.0           3.0         0.0   
              27            0.0          2.0           8.0         0.0   
...                         ...          ...           ...         ...   
1W        FW3 7018          0.0          0.0          10.0         0.0   
              7020          2.0          2.0           2.0         0.0   
              7022          3.0          0.0          21.0         0.0   
              7024          2.0          0.0          25.0         0.0   
              7025          0.0          1.0           3.0         0.0   

                                                            
PPI                P-ERK & c-MYC TEAD1 & YAP1 p-AKT & mTOR  
Condition FOV Id                                            
1M        FW1 1              0.0          2.0          1.0  
              9              0.0          1.0          3.0  
              25             0.0          0.0          0.0  
              26             0.0          0.0          0.0  
              27             0.0          0.0          0.0  
...                          ...          ...          ...  
1W        FW3 7018           5.0          1.0          2.0  
              7020          18.0          0.0          1.0  
              7022           7.0          0.0          5.0  
              7024          15.0          3.0          6.0  
              7025           0.0          6.0          1.0  

[43000 rows x 15 columns]

In [31]:
df_count_cyto_nuclei

Nuclei                                          
PPI                CylinE & CDK2 Mcl-1 & BAK P-ERK & c-MYC TEAD1 & YAP1   
Condition FOV Id                                                          
1M        FW1 1              0.0         0.0           0.0          0.0  \
              9              1.0         0.0           0.0          1.0   
              25             0.0         0.0           0.0          0.0   
              26             0.0         0.0           0.0          4.0   
              27             1.0         0.0           0.0          0.0   
...                          ...         ...           ...          ...   
1W        FW3 7018           0.0         0.0           0.0          0.0   
              7020           6.0         0.0           6.0          2.0   
              7022           2.0         0.0           2.0          3.0   
              7024          10.0         0.0          11.0          2.0   
              7025           0.0         0.0           0.0          0.0   

                                         Cyto                             
PPI                p-AKT & mTOR CylinE & CDK2 Mcl-1 & BAK P-ERK & c-MYC   
Condition FOV Id                                                          
1M        FW1 1             0.0           4.0         0.0           0.0  \
              9             0.0           1.0         0.0           0.0   
              25            4.0           2.0         0.0           0.0   
              26            0.0           3.0         0.0           0.0   
              27            2.0           8.0         0.0           0.0   
...                         ...           ...         ...           ...   
1W        FW3 7018          0.0          10.0         0.0           5.0   
              7020          2.0           2.0         0.0          18.0   
              7022          0.0          21.0         0.0           7.0   
              7024          0.0          25.0         0.0          15.0   
              7025          1.0           3.0         0.0           0.0   

                                              
PPI                TEAD1 & YAP1 p-AKT & mTOR  
Condition FOV Id                              
1M        FW1 1             2.0          1.0  
              9             1.0          3.0  
              25            0.0          0.0  
              26            0.0          0.0  
              27            0.0          0.0  
...                         ...          ...  
1W        FW3 7018          1.0          2.0  
              7020          0.0          1.0  
              7022          0.0          5.0  
              7024          3.0          6.0  
              7025          6.0          1.0  

[43000 rows x 10 columns]

In [32]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True), 
}

In [33]:
# Run model on cell count
X = df_count_cell
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cell', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.78244
auc,0.76657
b_accuracy,0.76657
f1,0.71962


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.78988
auc,0.77513
b_accuracy,0.77513
f1,0.7321


...

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.7914
auc,0.77628
b_accuracy,0.77628
f1,0.73272


wandb: 
wandb: Plotting SVM_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.78907
auc,0.77225
b_accuracy,0.77225
f1,0.72606


wandb: 
wandb: Plotting SVM_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.79488
auc,0.77872
b_accuracy,0.77872
f1,0.73474


In [34]:
# Run model on cell count
X = df_count_cyto_nuclei
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cyto_nuclei', name=model_name+f'_cyto_nuclei_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cyto_nuclei', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.78419
auc,0.7686
b_accuracy,0.7686
f1,0.72232


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.79535
auc,0.7812
b_accuracy,0.7812
f1,0.74003


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


...

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.80093
auc,0.78651
b_accuracy,0.78651
f1,0.74577


wandb: 
wandb: Plotting SVM_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.79372
auc,0.77878
b_accuracy,0.77878
f1,0.73554


wandb: 
wandb: Plotting SVM_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.79581
auc,0.78067
b_accuracy,0.78067
f1,0.73791
